In [ ]:
# import replicate

# input = {
#     "prompt": "Tina has one brother and one sister. How many sisters do Tina's siblings have?",
#     "max_tokens": 1024,
#     "temperature": 0
# }

# for event in replicate.stream(
#     "meta/meta-llama-3.1-405b-instruct",
#     input=input
# ):
#     print(event, end="")
# #=> "Tina has one brother and one sister. From the brother's ...

# English

In [1]:
import os
import time
import pandas as pd
import json
import re
from dotenv import load_dotenv
import replicate

# Load environment variables from .env file
load_dotenv()

# Set Replicate model and API key
model = "meta/meta-llama-3.1-405b-instruct"

# Load data
language_family = "english"
input_file_path = f'../../JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_direct_inference.csv"
summary_file_path = f"{language_family}_{model}_direct_inference_summary.json"

# Ensure the directory exists
output_dir = os.path.dirname(output_file_path)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

def get_replicate_response(claim):
    user_prompt = f"'{claim}' Is this claim 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'? You must always make sure your final response is prefixed with 'Final Answer:' followed by either 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'."

    retries = 0
    while True:
        try:
            input_data = {
                "prompt": user_prompt,
                "max_tokens": 1024,
                "temperature": 0
            }
            response = replicate.run(model, input=input_data)
            if isinstance(response, list):
                return ''.join(response)
            else:
                return response
        except Exception as e:
            if retries < 3:
                retries += 1
                wait_time = 4
                print(f"Error occurred: {e}. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                raise e

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    with open(summary_file_path, 'r', encoding='utf-8') as file:
        summary = json.load(file)
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Function to clean the output text
def clean_output(output):
    # Remove non-alphanumeric characters except spaces
    cleaned_output = re.sub(r'[^a-zA-Z\s]', '', output)
    return cleaned_output

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    output = get_replicate_response(claim)
    
    # Print the model's output
    print(f"Model Output: {output}")
    
    # Clean the output
    cleaned_output = clean_output(output)
    
    # Extract final answer from the cleaned output
    final_answer = None
    if "final answer true" in cleaned_output.lower():
        final_answer = "true"
    elif "final answer false" in cleaned_output.lower():
        final_answer = "false"
    elif "final answer mostly true" in cleaned_output.lower():
        final_answer = "mostly true"
    elif "final answer mostly false" in cleaned_output.lower():
        final_answer = "mostly false"
    elif "final answer half true" in cleaned_output.lower():
        final_answer = "half true"
    
    # Determine correctness or inconclusiveness
    if final_answer is None:
        print("Inconclusive response")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response")
            summary['correct'] += 1
        else:
            print("Wrong response")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'output': output,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


/Users/aryan/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Model Output: To determine the accuracy of the claim, let's break down the steps to verify it:

1. **Identify the Source and Context**: The claim seems to be from a news article or a political statement about New Jersey's job growth in May. Without a specific source or date, we'll consider this as a general claim that needs verification against available economic data.

2. **Understand the Claim**: The claim is that New Jersey's job growth in a particular May accounts for 25% of all jobs created in the country for that month.

3. **Verify Against Data**: To verify this claim, we would typically look at employment data from reputable sources such as the Bureau of Labor Statistics (BLS), which tracks job growth state by state and nationally. However, without access to real-time data or the ability to fetch the latest statistics, we must consider the plausibility based on general trends and the size of New Jersey's economy relative to the entire U.S.

4. **Consider the Size of New Jersey'

# Romance

In [2]:
import os
import time
import pandas as pd
import json
import re
from dotenv import load_dotenv
import replicate

# Load environment variables from .env file
load_dotenv()

# Set Replicate model and API key
model = "meta/meta-llama-3.1-405b-instruct"

# Define the prompting method and language family
language_family = "romance"
method = "direct_inference"

# Load data
input_file_path = f'../../JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

# Ensure the directory exists
output_dir = os.path.dirname(output_file_path)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

# Language-specific user prompts and final answer mappings
prompts = {
    "es": "'{claim}' ¿Es esta afirmación «Verdadera», «Casi verdadera», «Medio verdadera», «Casi falsa» o «Falsa»? Debe asegurarse de que su respuesta final comience con «Respuesta final:» seguida de «Verdadero», «Casi verdadero», «Medio verdadero», «Casi falso» o «Falso».",
    "it": "'{claim}' Questa affermazione è “vera”, “quasi vera”, “mezza vera”, “quasi falsa” o “falsa”? Assicurati che la tua risposta finale inizi con “Risposta finale:” seguita da “Vero”, “Quasi vero”, “Mezzo vero”, “Quasi falso” o “Falso”.",
    "pt": "'{claim}' Esta afirmação é “Verdadeira”, “Quase Verdadeira”, “Meia Verdadeira”, “Quase Falsa” ou “Falsa”? Certifique-se de que sua resposta final comece com “Resposta Final:” seguida de “Verdadeiro”, “Quase Verdadeiro”, “Meio Verdadeiro”, “Quase Falso” ou “Falso”.",
    "fr": "'{claim}' Cette affirmation est-elle « vraie », « en grande partie vraie », « à moitié vraie », « en grande partie fausse » ou « fausse » ? Vous devez vous assurer que votre réponse finale commence par « Réponse finale: » suivie de « Vrai », « En grande partie vrai », « À moitié vrai », « En grande partie faux » ou « Faux »."
}

final_answer_mappings = {
    "es": {
        "respuesta final: verdadero": "true", "respuesta final: verdadera": "true",
        "respuesta final: casi verdadero": "mostly true", "respuesta final: casi verdadera": "mostly true",
        "respuesta final: medio verdadero": "half true", "respuesta final: medio verdadera": "half true",
        "respuesta final: casi falso": "mostly false", "respuesta final: casi falsa": "mostly false",
        "respuesta final: falso": "false", "respuesta final: falsa": "false",
        "verdadero": "true", "verdadera": "true",
        "casi verdadero": "mostly true", "casi verdadera": "mostly true",
        "medio verdadero": "half true", "medio verdadera": "half true",
        "casi falso": "mostly false", "casi falsa": "mostly false",
        "falso": "false", "falsa": "false"
    },
    "it": {
        "risposta finale: vero": "true", "risposta finale: vera": "true",
        "risposta finale: quasi vero": "mostly true", "risposta finale: quasi vera": "mostly true",
        "risposta finale: mezzo vero": "half true", "risposta finale: mezza vera": "half true",
        "risposta finale: quasi falso": "mostly false", "risposta finale: quasi falsa": "mostly false",
        "risposta finale: falso": "false", "risposta finale: falsa": "false",
        "vero": "true", "vera": "true",
        "quasi vero": "mostly true", "quasi vera": "mostly true",
        "mezzo vero": "half true", "mezza vera": "half true",
        "quasi falso": "mostly false", "quasi falsa": "mostly false",
        "falso": "false", "falsa": "false"
    },
    "pt": {
        "resposta final: verdadeiro": "true", "resposta final: verdadeira": "true",
        "resposta final: quase verdadeiro": "mostly true", "resposta final: quase verdadeira": "mostly true",
        "resposta final: meio verdadeiro": "half true", "resposta final: meia verdadeira": "half true",
        "resposta final: quase falso": "mostly false", "resposta final: quase falsa": "mostly false",
        "resposta final: falso": "false", "resposta final: falsa": "false",
        "verdadeiro": "true", "verdadeira": "true",
        "quase verdadeiro": "mostly true", "quase verdadeira": "mostly true",
        "meio verdadeiro": "half true", "meia verdadeira": "half true",
        "quase falso": "mostly false", "quase falsa": "mostly false",
        "falso": "false", "falsa": "false"
    },
    "fr": {
        "réponse finale: vrai": "true", "réponse finale: vraie": "true",
        "réponse finale: plus ou moins vrai": "mostly true", "réponse finale: plus ou moins vraie": "mostly true",
        "réponse finale: moitié vrai": "half true", "réponse finale: moitié vraie": "half true",
        "réponse finale: plus ou moins faux": "mostly false", "réponse finale: plus ou moins fausse": "mostly false",
        "réponse finale: faux": "false", "réponse finale: fausse": "false",
        "vrai": "true", "vraie": "true",
        "plus ou moins vrai": "mostly true", "plus ou moins vraie": "mostly true",
        "moitié vrai": "half true", "moitié vraie": "half true",
        "plus ou moins faux": "mostly false", "plus ou moins fausse": "mostly false",
        "faux": "false", "fausse": "false"
    }
}

def get_replicate_response(claim, language, model=model):
    user_prompt = prompts[language].format(claim=claim)
    retries = 0
    while True:
        try:
            input_data = {
                "prompt": user_prompt,
                "max_tokens": 1024,
                "temperature": 0
            }
            response = replicate.run(model, input=input_data)
            if isinstance(response, list):
                return ''.join(response)
            else:
                return response
        except Exception as e:
            if retries < 3:
                retries += 1
                wait_time = 4
                print(f"Error occurred: {e}. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                raise e

def clean_output(output):
    cleaned_output = re.sub(r'[^a-zA-Z\s:]', '', output)
    return cleaned_output.lower()

def extract_final_answer(cleaned_output, language):
    try:
        final_answer = re.search(r'(respuesta final:|risposta finale:|resposta final:|réponse finale:)\s*([\w\s]+)', cleaned_output)
        if final_answer:
            response = final_answer.group(2).strip()
            for keyword, answer in final_answer_mappings[language].items():
                if keyword in response:
                    return answer
        else:
            for keyword, answer in final_answer_mappings[language].items():
                if keyword in cleaned_output:
                    return answer
    except Exception as e:
        print(f"Error extracting final answer: {e}")
    return None

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    with open(summary_file_path, 'r', encoding='utf-8') as file:
        summary = json.load(file)
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    if language not in prompts:  # Skip unsupported languages
        continue

    output = get_replicate_response(claim, language)
    
    print(f"Model Output: {output}")
    
    cleaned_output = clean_output(output)
    final_answer = extract_final_answer(cleaned_output, language)
    
    if final_answer is None:
        print("Inconclusive response")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response")
            summary['correct'] += 1
        else:
            print("Wrong response")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'output': output,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


Model Output: Réponse finale: Vrai.
Final Answer: True, Actual Answer: False
Wrong response
Model Output: Réponse finale: Vrai.
Final Answer: True, Actual Answer: True
Correct response
Model Output: Risposta finale: Vero.
Final Answer: True, Actual Answer: Mostly true
Wrong response
Model Output: Risposta finale: Mezzo vero.
Final Answer: True, Actual Answer: True
Correct response
Model Output: Respuesta final: Falso
Final Answer: False, Actual Answer: True
Wrong response
Model Output: Réponse finale: Vrai.
Final Answer: True, Actual Answer: Half true
Wrong response
Model Output: Risposta finale: Mezzo vero.
Final Answer: True, Actual Answer: True
Correct response
Model Output: Respuesta final: Falso
Final Answer: False, Actual Answer: Half true
Wrong response
Model Output: Resposta Final: Verdadeira
Final Answer: True, Actual Answer: Mostly true
Wrong response
Model Output: Risposta finale: Vero.
Final Answer: True, Actual Answer: Mostly true
Wrong response
Model Output: Respuesta fin

# Indo-Aryan

In [3]:
import os
import time
import pandas as pd
import json
import re
from dotenv import load_dotenv
import replicate

# Load environment variables from .env file
load_dotenv()

# Set Replicate model and API key
model = "meta/meta-llama-3.1-405b-instruct"

# Define the prompting method and language family
language_family = "indo_aryan"
method = "direct_inference"

# Load data
input_file_path = f'../../JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

# Ensure the directory exists
output_dir = os.path.dirname(output_file_path)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

# Language-specific user prompts and final answer mappings
prompts = {
    "hi": "'{claim}' क्या यह कथन «सत्य», «अधिकांशतः सत्य», «आधा सत्य», «अधिकांशतः असत्य» या «असत्य» है? आपको हमेशा यह सुनिश्चित करना चाहिए कि आपका अंतिम उत्तर «अंतिम उत्तर:» के साथ शुरू हो और उसके बाद «सत्य», «अधिकांशतः सत्य», «आधा सत्य», «अधिकांशतः असत्य» या «असत्य» हो।",
    "bn": "'{claim}' এই বক্তব্যটি «সত্য», «অধিকাংশ সত্য», «অর্ধসত্য», «অধিকাংশ মিথ্যা» বা «মিথ্যা» কি? সর্বদা নিশ্চিত করুন যে আপনার চূড়ান্ত উত্তর «চূড়ান্ত উত্তর:» দিয়ে শুরু হয় এবং তারপরে «সত্য», «অধিকাংশ সত্য», «অর্ধসত্য», «অধিকাংশ মিথ্যা» বা «মিথ্যা»।",
    "pa": "'{claim}' ਕੀ ਇਹ ਬਿਆਨ «ਸੱਚ», «ਜਿਆਦਾਤਰ ਸੱਚ», «ਅੱਧਾ ਸੱਚ», «ਜਿਆਦਾਤਰ ਝੂਠ» ਜਾਂ «ਝੂਠ» ਹੈ? ਤੁਹਾਨੂੰ ਹਮੇਸ਼ਾਂ ਇਹ ਯਕੀਨੀ ਬਣਾਉਣਾ ਚਾਹੀਦਾ ਹੈ ਕਿ ਤੁਹਾਡਾ ਅੰਤਿਮ ਜਵਾਬ «ਅੰਤਿਮ ਜਵਾਬ:» ਨਾਲ ਸ਼ੁਰੂ ਹੋਵੇ ਅਤੇ ਫਿਰ «ਸੱਚ», «ਜਿਆਦਾਤਰ ਸੱਚ», «ਅੱਧਾ ਸੱਚ», «ਜਿਆਦਾਤਰ ਝੂਠ» ਜਾਂ «ਝੂਠ» ਹੋਵੇ।",
    "gu": "'{claim}' શું આ નિવેદન «સત્ય», «મોટાભાગનું સત્ય», «અડધું સત્ય», «મોટાભાગનું ખોટું» કે «ખોટું» છે? તમે હંમેશાં ખાતરી કરવી જોઈએ કે તમારો અંતિમ જવાબ «અંતિમ જવાબ:» થી શરૂ થાય અને પછી «સત્ય», «મોટાભાગનું સત્ય», «અડધું સત્ય», «મોટાભાગનું ખોટું» કે «ખોટું» આવે।",
    "mr": "'{claim}' हे विधान «सत्य», «जास्तीत जास्त सत्य», «अर्धसत्य», «जास्तीत जास्त असत्य» किंवा «असत्य» आहे का? तुम्ही नेहमी हे सुनिश्चित केले पाहिजे की तुमचे अंतिम उत्तर «अंतिम उत्तर:» ने सुरू होते आणि त्यानंतर «सत्य», «जास्तीत जास्त सत्य», «अर्धसत्य», «जास्तीत जास्त असत्य» किंवा «असत्य» येते."
}

final_answer_mappings = {
    "hi": {
        "अंतिम उत्तर: सत्य": "true",
        "अंतिम उत्तर: अधिकांशतः सत्य": "mostly true",
        "अंतिम उत्तर: आधा सत्य": "half true",
        "अंतिम उत्तर: अधिकांशतः असत्य": "mostly false",
        "अंतिम उत्तर: असत्य": "false",
        "सत्य": "true",
        "अधिकांशतः सत्य": "mostly true",
        "आधा सत्य": "half true",
        "अधिकांशतः असत्य": "mostly false",
        "असत्य": "false"
    },
    "bn": {
        "চূড়ান্ত উত্তর: সত্য": "true",
        "চূড়ান্ত উত্তর: অধিকাংশ সত্য": "mostly true",
        "চূড়ান্ত উত্তর: অর্ধসত্য": "half true",
        "চূড়ান্ত উত্তর: অধিকাংশ মিথ্যা": "mostly false",
        "চূড়ান্ত উত্তর: মিথ্যা": "false",
        "সত্য": "true",
        "অধিকাংশ সত্য": "mostly true",
        "অর্ধসত্য": "half true",
        "অধিকাংশ মিথ্যা": "mostly false",
        "মিথ্যা": "false"
    },
    "pa": {
        "ਅੰਤਿਮ ਜਵਾਬ: ਸੱਚ": "true",
        "ਅੰਤਿਮ ਜਵਾਬ: ਜਿਆਦਾਤਰ ਸੱਚ": "mostly true",
        "ਅੰਤਿਮ ਜਵਾਬ: ਅੱਧਾ ਸੱਚ": "half true",
        "ਅੰਤਿਮ ਜਵਾਬ: ਜਿਆਦਾਤਰ ਝੂਠ": "mostly false",
        "ਅੰਤਿਮ ਜਵਾਬ: ਝੂਠ": "false",
        "ਸੱਚ": "true",
        "ਜਿਆਦਾਤਰ ਸੱਚ": "mostly true",
        "ਅੱਧਾ ਸੱਚ": "half true",
        "ਜਿਆਦਾਤਰ ਝੂਠ": "mostly false",
        "ਝੂਠ": "false"
    },
    "gu": {
        "અંતિમ જવાબ: સત્ય": "true",
        "અંતિમ જવાબ: મોટાભાગનું સત્ય": "mostly true",
        "અંતિમ જવાબ: અડધું સત્ય": "half true",
        "અંતિમ જવાબ: મોટાભાગનું ખોટું": "mostly false",
        "અંતિમ જવાબ: ખોટું": "false",
        "સત્ય": "true",
        "મોટાભાગનું સત્ય": "mostly true",
        "અડધું સત્ય": "half true",
        "મોટાભાગનું ખોટું": "mostly false",
        "ખોટું": "false"
    },
    "mr": {
        "अंतिम उत्तर: सत्य": "true",
        "अंतिम उत्तर: जास्तीत जास्त सत्य": "mostly true",
        "अंतिम उत्तर: अर्धसत्य": "half true",
        "अंतिम उत्तर: जास्तीत जास्त असत्य": "mostly false",
        "अंतिम उत्तर: असत्य": "false",
        "सत्य": "true",
        "जास्तीत जास्त सत्य": "mostly true",
        "अर्धसत्य": "half true",
        "जास्तीत जास्त असत्य": "mostly false",
        "असत्य": "false"
    }
}

def get_replicate_response(claim, language, model=model):
    user_prompt = prompts[language].format(claim=claim)
    retries = 0
    while True:
        try:
            input_data = {
                "prompt": user_prompt,
                "max_tokens": 1024,
                "temperature": 0
            }
            response = replicate.run(model, input=input_data)
            if isinstance(response, list):
                return ''.join(response)
            else:
                return response
        except Exception as e:
            if retries < 3:
                retries += 1
                wait_time = 4
                print(f"Error occurred: {e}. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                raise e

def clean_output(output):
    cleaned_output = re.sub(r'[^a-zA-Z\s:]', '', output)
    return cleaned_output.lower()

def extract_final_answer(cleaned_output, language):
    try:
        final_answer = re.search(r'(अंतिम उत्तर:|চূড়ান্ত উত্তর:|ਅੰਤਿਮ ਜਵਾਬ:|અંતિમ જવાબ:|अंतिम उत्तर:)\s*([\w\s]+)', cleaned_output)
        if final_answer:
            response = final_answer.group(2).strip()
            for keyword, answer in final_answer_mappings[language].items():
                if keyword in response:
                    return answer
        else:
            for keyword, answer in final_answer_mappings[language].items():
                if keyword in cleaned_output:
                    return answer
    except Exception as e:
        print(f"Error extracting final answer: {e}")
    return None

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    with open(summary_file_path, 'r', encoding='utf-8') as file:
        summary = json.load(file)
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    if language not in prompts:  # Skip unsupported languages
        continue

    output = get_replicate_response(claim, language)
    
    print(f"Model Output: {output}")
    
    cleaned_output = clean_output(output)
    final_answer = extract_final_answer(cleaned_output, language)
    
    if final_answer is None:
        print("Inconclusive response")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response")
            summary['correct'] += 1
        else:
            print("Wrong response")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'output': output,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


Model Output: চূড়ান্ত উত্তর: সত্য
Inconclusive response
Model Output: अंतिम उत्तर: सत्य
Inconclusive response
Model Output: अंतिम उत्तर: असत्य
Inconclusive response
Model Output: ਅੰਤਿਮ ਜਵਾਬ: ਝੂਠ
Inconclusive response
Model Output: চূড়ান্ত উত্তর: সত্য
Inconclusive response
Model Output: अंतिम उत्तर: सत्य
Inconclusive response
Model Output: চূড়ান্ত উত্তর: মিথ্যা
Inconclusive response
Model Output: अंतिम उत्तर: सत्य
Inconclusive response
Model Output: अंतिम उत्तर: असत्य
Inconclusive response
Model Output: চূড়ান্ত উত্তর: সত্য
Inconclusive response
Model Output: চূড়ান্ত উত্তর: সত্য
Inconclusive response
Model Output: চূড়ান্ত উত্তর: সত্য
Inconclusive response
Model Output: চূড়ান্ত উত্তর: সত্য।
Inconclusive response
Model Output: চূড়ান্ত উত্তর: সত্য।
Inconclusive response
Model Output: अंतिम उत्तर: असत्य
Inconclusive response
Model Output: अंतिम उत्तर: असत्य
Inconclusive response
Model Output: अंतिम उत्तर: असत्य
Inconclusive response
Model Output: চূড়ান্ত উত্তর: মিথ্যা
Inconclusive r

# Kartvelian

In [4]:
import os
import time
import pandas as pd
import json
import re
from dotenv import load_dotenv
import replicate

# Load environment variables from .env file
load_dotenv()

# Set Replicate model and API key
model = "meta/meta-llama-3.1-405b-instruct"

# Define the prompting method and language family
language_family = "kartvelian"
method = "direct_inference"

# Load data
input_file_path = f'../../JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

# Ensure the directory exists
output_dir = os.path.dirname(output_file_path)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

# Language-specific user prompts and final answer mappings
prompts = {
    "ka": "'{claim}' არის ეს პრეტენზია «მართალია», «ძირითადად მართალია», «ნახევრად მართალია», «ძირითადად მცდარი» ან «მცდარი»? თქვენ ყოველთვის უნდა დარწმუნდეთ, რომ თქვენს საბოლოო პასუხს აქვს პრეფიქსი „საბოლოო პასუხი:“ რასაც მოჰყვება „მართალი“, „ძირითადად მართალია“, „ნახევრად მართალია“, „ძირითადად მცდარი“ ან „მცდარი“."
}

final_answer_mappings = {
    "ka": {
        "საბოლოო პასუხი: ჭეშმარიტი": "true",
        "საბოლოო პასუხი: უმეტესად ჭეშმარიტი": "mostly true",
        "საბოლოო პასუხი: ნახევრად ჭეშმარიტი": "half true",
        "საბოლოო პასუხი: უმეტესად ცრუ": "mostly false",
        "საბოლოო პასუხი: ცრუ": "false",
        "ჭეშმარიტი": "true",
        "უმეტესად ჭეშმარიტი": "mostly true",
        "ნახევრად ჭეშმარიტი": "half true",
        "უმეტესად ცრუ": "mostly false",
        "ცრუ": "false"
    }
}

def get_replicate_response(claim, language, model=model):
    user_prompt = prompts[language].format(claim=claim)
    retries = 0
    while True:
        try:
            input_data = {
                "prompt": user_prompt,
                "max_tokens": 1024,
                "temperature": 0
            }
            response = replicate.run(model, input=input_data)
            if isinstance(response, list):
                return ''.join(response)
            else:
                return response
        except Exception as e:
            if retries < 3:
                retries += 1
                wait_time = 4
                print(f"Error occurred: {e}. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                raise e

def clean_output(output):
    cleaned_output = re.sub(r'[^a-zA-Zა-ჰ\s:]', '', output)
    return cleaned_output.lower()

def extract_final_answer(cleaned_output, language):
    try:
        final_answer = None
        # First, look for the exact prefix "საბოლოო პასუხი:"
        match = re.search(r'საბოლოო პასუხი[:]*\s*([\w\s]+)', cleaned_output)
        if match:
            response = match.group(1).strip()
            for keyword, answer in final_answer_mappings[language].items():
                if keyword in response:
                    final_answer = answer
                    break
        if not final_answer:
            # If no exact match found, look for keywords directly in the cleaned output
            for keyword, answer in final_answer_mappings[language].items():
                if keyword in cleaned_output:
                    final_answer = answer
                    break
        return final_answer
    except Exception as e:
        print(f"Error extracting final answer: {e}")
    return None

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    try:
        with open(summary_file_path, 'r', encoding='utf-8') as file:
            file_content = file.read().strip()
            if file_content:  # Check if the file is not empty
                summary = json.loads(file_content)
            else:
                raise json.JSONDecodeError("Empty file", file_content, 0)
    except (json.JSONDecodeError, FileNotFoundError):
        # Initialize the summary if the file is empty or not valid JSON
        summary = {
            'correct': 0,
            'wrong': 0,
            'inconclusive': 0,
            'total': 0,
            'languages': {}
        }
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    if language not in prompts:  # Skip unsupported languages
        continue

    output = get_replicate_response(claim, language)
    
    print(f"Model Output: {output}")
    
    cleaned_output = clean_output(output)
    final_answer = extract_final_answer(cleaned_output, language)
    
    if final_answer is None:
        print("Inconclusive response")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response")
            summary['correct'] += 1
        else:
            print("Wrong response")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'output': output,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


Model Output: საბოლოო პასუხი: მცდარი.
Inconclusive response
Model Output: საბოლოო პასუხი: მცდარი.
Inconclusive response
Model Output: საბოლოო პასუხი: მართალია.
Inconclusive response
Model Output: საბოლოო პასუხი: მართალია.
Inconclusive response
Model Output: საბოლოო პასუხი: მართალია.
Inconclusive response
Model Output: საბოლოო პასუხი: მართალია.
Inconclusive response
Model Output: საბოლოო პასუხი: მართალია.
Inconclusive response
Model Output: საბოლოო პასუხი: მართალია.
Inconclusive response
Model Output: საბოლოო პასუხი: მართალია.
Inconclusive response
Model Output: საბოლოო პასუხი: მართალია.
Inconclusive response
Model Output: საბოლოო პასუხი: მართალია.
Inconclusive response
Model Output: საბოლოო პასუხი: ძირითადად მართალია.
Inconclusive response
Model Output: საბოლოო პასუხი: მართალია.
Inconclusive response
Model Output: საბოლოო პასუხი: მართალია.
Inconclusive response
Model Output: საბოლოო პასუხი: მცდარი.
Inconclusive response
Model Output: საბოლოო პასუხი: მართალია.
Inconclusive response
Mode

# Slavic

In [ ]:
import os
import time
import pandas as pd
import json
import re
from dotenv import load_dotenv
import replicate

# Load environment variables from .env file
load_dotenv()

# Set Replicate model and API key
model = "meta/meta-llama-3.1-405b-instruct"

# Define the prompting method and language family
language_family = "slavic"
method = "direct_inference"

# Load data
input_file_path = f'../../JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

# Ensure the directory exists
output_dir = os.path.dirname(output_file_path)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

# Language-specific user prompts and final answer mappings
prompts = {
    "ru": "'{claim}' является ли это утверждение «правдивым», «в основном правдивым», «наполовину правдивым», «в основном ложным» или «ложным»? Вы должны всегда убедиться, что ваш окончательный ответ начинается с «Окончательный ответ:» и далее следует «Правда», «В основном правда», «Наполовину правда», «В основном ложь» или «Ложь».",
    "pl": "'{claim}' czy to stwierdzenie jest „prawdziwe”, „przeważnie prawdziwe”, „w połowie prawdziwe”, „przeważnie fałszywe” czy „fałszywe”? Musisz zawsze upewnić się, że twoja ostateczna odpowiedź zaczyna się od „Ostateczna odpowiedź:” a następnie „Prawda”, „Przeważnie prawda”, „W połowie prawda”, „Przeważnie fałsz” lub „Fałsz”.",
    "sr": "'{claim}' да ли је ова изјава „истинита”, „углавном истина”, „полуистинита”, „углавном лажна” или „лажна”? Морате увек осигурати да ваш коначан одговор почиње са „Коначан одговор:” и затим следи „Истина”, „Углавном истина”, „Полуистина”, „Углавном лаж” или „Лаж”."
}

final_answer_mappings = {
    "ru": {
        "окончательный ответ: правда": "true",
        "окончательный ответ: в основном правда": "mostly true",
        "окончательный ответ: наполовину правда": "half true",
        "окончательный ответ: в основном ложь": "mostly false",
        "окончательный ответ: ложь": "false",
        "правда": "true",
        "в основном правда": "mostly true",
        "наполовину правда": "half true",
        "в основном ложь": "mostly false",
        "ложь": "false"
    },
    "pl": {
        "ostateczna odpowiedź: prawda": "true",
        "ostateczna odpowiedź: przeważnie prawda": "mostly true",
        "ostateczna odpowiedź: w połowie prawda": "half true",
        "ostateczna odpowiedź: przeważnie fałsz": "mostly false",
        "ostateczna odpowiedź: fałsz": "false",
        "prawda": "true",
        "przeważnie prawda": "mostly true",
        "w połowie prawda": "half true",
        "przeważnie fałsz": "mostly false",
        "fałsz": "false"
    },
    "sr": {
        "коначан одговор: истина": "true",
        "коначан одговор: углавном истина": "mostly true",
        "коначан одговор: полуистина": "half true",
        "коначан одговор: углавном лаж": "mostly false",
        "коначан одговор: лаж": "false",
        "истина": "true",
        "углавном истина": "mostly true",
        "полуистина": "half true",
        "углавном лаж": "mostly false",
        "лаж": "false"
    }
}

def get_replicate_response(claim, language, model=model):
    user_prompt = prompts[language].format(claim=claim)
    retries = 0
    while True:
        try:
            input_data = {
                "prompt": user_prompt,
                "max_tokens": 1024,
                "temperature": 0
            }
            response = replicate.run(model, input=input_data)
            if isinstance(response, list):
                return ''.join(response)
            else:
                return response
        except Exception as e:
            if retries < 3:
                retries += 1
                wait_time = 4
                print(f"Error occurred: {e}. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                raise e

def clean_output(output):
    cleaned_output = re.sub(r'[^a-zA-Zа-яА-Я\s:]', '', output)
    return cleaned_output.lower()

def extract_final_answer(cleaned_output, language):
    try:
        final_answer = re.search(r'(окончательный ответ:|ostateczna odpowiedź:|коначан одговор:)\s*([\w\s]+)', cleaned_output)
        if final_answer:
            response = final_answer.group(2).strip()
            for keyword, answer in final_answer_mappings[language].items():
                if keyword in response:
                    return answer
        else:
            for keyword, answer in final_answer_mappings[language].items():
                if keyword in cleaned_output:
                    return answer
    except Exception as e:
        print(f"Error extracting final answer: {e}")
    return None

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    with open(summary_file_path, 'r', encoding='utf-8') as file:
        summary = json.load(file)
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    if language not in prompts:  # Skip unsupported languages
        continue

    output = get_replicate_response(claim, language)
    
    print(f"Model Output: {output}")
    
    cleaned_output = clean_output(output)
    final_answer = extract_final_answer(cleaned_output, language)
    
    if final_answer is None:
        print("Inconclusive response")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response")
            summary['correct'] += 1
        else:
            print("Wrong response")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'output': output,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


# Turkic

In [ ]:
import os
import time
import pandas as pd
import json
import re
from dotenv import load_dotenv
import replicate

# Load environment variables from .env file
load_dotenv()

# Set Replicate model and API key
model = "meta/meta-llama-3.1-405b-instruct"

# Define the prompting method and language family
language_family = "turkic"
method = "direct_inference"

# Load data
input_file_path = f'../../JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

# Ensure the directory exists
output_dir = os.path.dirname(output_file_path)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

# Language-specific user prompts and final answer mappings
prompts = {
    "tr": "'{claim}' bu ifade «doğru», «çoğunlukla doğru», «yarı doğru», «çoğunlukla yanlış» veya «yanlış» mı? «Nihai Cevap: Doğru», «Nihai Cevap: Çoğunlukla Doğru», «Nihai Cevap: Yarı Doğru», «Nihai Cevap: Çoğunlukla Yanlış» veya «Nihai Cevap: Yanlış» şeklinde cevap verin.",
    "az": "'{claim}' bu ifadə «doğru», «əsasən doğru», «yarı doğru», «əsasən yanlış» və ya «yanlış»dır? «Son Cavab: Doğru», «Son Cavab: Əsasən Doğru», «Son Cavab: Yarı Doğru», «Son Cavab: Əsasən Yanlış» və ya «Son Cavab: Yanlış» olaraq cavab verin."
}

final_answer_mappings = {
    "tr": {
        "nihai cevap: doğru": "true",
        "nihai cevap: çoğunlukla doğru": "mostly true",
        "nihai cevap: yarı doğru": "half true",
        "nihai cevap: çoğunlukla yanlış": "mostly false",
        "nihai cevap: yanlış": "false",
        "doğru": "true",
        "çoğunlukla doğru": "mostly true",
        "yarı doğru": "half true",
        "çoğunlukla yanlış": "mostly false",
        "yanlış": "false"
    },
    "az": {
        "son cavab: doğru": "true",
        "son cavab: əsasən doğru": "mostly true",
        "son cavab: yarı doğru": "half true",
        "son cavab: əsasən yanlış": "mostly false",
        "son cavab: yanlış": "false",
        "doğru": "true",
        "əsasən doğru": "mostly true",
        "yarı doğru": "half true",
        "əsasən yanlış": "mostly false",
        "yanlış": "false"
    }
}

def get_replicate_response(claim, language, model=model):
    user_prompt = prompts[language].format(claim=claim)
    retries = 0
    while True:
        try:
            input_data = {
                "prompt": user_prompt,
                "max_tokens": 1024,
                "temperature": 0
            }
            response = replicate.run(model, input=input_data)
            if isinstance(response, list):
                return ''.join(response)
            else:
                return response
        except Exception as e:
            if retries < 3:
                retries += 1
                wait_time = 4
                print(f"Error occurred: {e}. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                raise e

def clean_output(output):
    cleaned_output = re.sub(r'[^a-zA-ZəƏıİşŞöÖüÜğĞçÇ\s:]', '', output)
    return cleaned_output.lower()

def extract_final_answer(cleaned_output, language):
    try:
        final_answer = re.search(r'(nihai cevap:|son cavab:)\s*([\w\s]+)', cleaned_output)
        if final_answer:
            response = final_answer.group(2).strip()
            for keyword, answer in final_answer_mappings[language].items():
                if keyword in response:
                    return answer
        else:
            for keyword, answer in final_answer_mappings[language].items():
                if keyword in cleaned_output:
                    return answer
    except Exception as e:
        print(f"Error extracting final answer: {e}")
    return None

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    with open(summary_file_path, 'r', encoding='utf-8') as file:
        summary = json.load(file)
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    if language not in prompts:  # Skip unsupported languages
        continue

    output = get_replicate_response(claim, language)
    
    print(f"Model Output: {output}")
    
    cleaned_output = clean_output(output)
    final_answer = extract_final_answer(cleaned_output, language)
    
    if final_answer is None:
        print("Inconclusive response")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response")
            summary['correct'] += 1
        else:
            print("Wrong response")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'output': output,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")
